In [ ]:
# Install required packages and import necessary libraries
!pip install s3fs==2024.9.0 wandb==0.18.0 -qqq

# s3fs for accessing S3 MinIO storage, os for system operations, PIL for image handling
import s3fs
import os
from PIL import Image

# Set up connection to S3 MinIO bucket with provided credentials and endpoint
fs = s3fs.S3FileSystem(
    key='KEY_ID',
    secret='KEY_SECRET',
    client_kwargs={'endpoint_url': 'ENDPOINT'}
)

# Initialize and login to Weights and Biases (wandb) for experiment tracking
import wandb
wandb.login()

In [ ]:
# Define the local paths where training, validation, and test datasets will be stored
local_training_dir = '/home/jovyan/data/training'
local_validation_dir = '/home/jovyan/data/validation'
local_test_dir = '/home/jovyan/data/test'

# Create the directories if they don't already exist
os.makedirs(local_training_dir, exist_ok=True)
os.makedirs(local_validation_dir, exist_ok=True)
os.makedirs(local_test_dir, exist_ok=True)

# List the files in the training, validation, and test directories in the S3 MinIO bucket
training_files = fs.ls('datasets/coco/1/datasets/training')
validation_files = fs.ls('datasets/coco/1/datasets/validation')
test_files = fs.ls('datasets/coco/1/datasets/test')

# Download each file from the S3 bucket and save it to the corresponding local directory for training
for file in training_files:
    local_file_path = os.path.join(local_training_dir, os.path.basename(file))
    fs.get(file, local_file_path)

# Download each file from the S3 bucket and save it to the corresponding local directory for validation
for file in validation_files:
    local_file_path = os.path.join(local_validation_dir, os.path.basename(file))
    fs.get(file, local_file_path)

# Download each file from the S3 bucket and save it to the corresponding local directory for testing
for file in test_files:
    local_file_path = os.path.join(local_test_dir, os.path.basename(file))
    fs.get(file, local_file_path)

In [ ]:
# Clone the YOLOv5-Lite-Fork repository from GitHub, which contains the code and configuration for the YOLOv5-Lite model
!git clone https://github.com/jurski-eu/YOLOv5-Lite-Fork.git

# Change the current working directory to the YOLOv5-Lite-Fork directory
os.chdir('./YOLOv5-Lite-Fork')

# Install the necessary dependencies for YOLOv5-Lite from the requirements.txt file in the cloned repository
!pip install -r ./requirements.txt -qqq

In [ ]:
# Run the YOLOv5-Lite training script with the specified configuration:
# --data: Path to the dataset configuration file (person.yaml)
# --cfg: Path to the model configuration file (v5Lite-s.yaml)
# --weights: Start training from scratch with no pre-trained weights
# --batch-size: Set the batch size to 1000
# --img-size: Set the image size for training to 320x320 pixels
!python ./train.py --data ../person.yaml --cfg ./models/v5Lite-s.yaml --weights '' --batch-size 1000 --img-size 320